# Data Augmenation

In [ ]:
import os
import cv2
import sys
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = random.seed(1243)

#Tensorflow background
import tensorflow as tf

## Data Driven Approach

**Splitting the dataset into #3 parts:**
- Training  
- Testing  
- Evaluation  

**Then augmentate training part**

In [ ]:
pwd

In [ ]:
# Path to preprocessed dataset
path_data = r'C:\Users\grofc\Desktop\Preprocessed'
data_len = None

In [ ]:
# Set size of evaluation/test part
test = 200
evaluation = 400

**Load preprocessed dataset**

In [ ]:
#Function loads the image paths

def load_data(path):
    
    global data_len
    os.chdir(path)
    data = [os.path.abspath(f) for f in  os.listdir(path_data)]
    data_len = len(data)
    return data   

In [ ]:
# Load the preprocessed data
data = load_data(path_data)
print(f'Dataset images: {data_len}')

In [ ]:
# Set destination directories
dir_dst = r'C:\Users\grofc\Desktop\Augmented'
dir_train = dir_dst + '\Train'
dir_test = dir_dst + '\Test'
dir_evaluation = dir_dst + '\Evaluation'

try:
    os.makedirs(dir_train)
    os.makedirs(dir_test)
    os.makedirs(dir_evaluation)
except:
    print('Directories already exists or failed'
         + ' to create!')


**Initialize Random generator**

In [ ]:
# Function to random choose indices of preprocessed dataset

def random_indices(length, num_to):
    indices = set()

    while len(indices) < num_to:
        num = random.randint(0, length-1)
        indices.add(num)

    indices = list(indices)
    print('Length of list is: ', len(indices))
    return indices

In [ ]:
# Function moves data directories to destination acc. indices
def move_images(data, indices, dst):
    
    if data is None or len(indices) == 0:
        return
    
    for i in indices:
        shutil.move(data[i], dst)

**Splitting into parts**

In [ ]:
#Firstly split into test part
indices_test = random_indices(data_len, test)
move_images(data, indices_test, dir_test)

#Load and split into evaluation dataset
data = load_data(path_data)
indices_eval = random_indices(data_len, evaluation)
move_images(data, indices_eval, dir_evaluation)

In [ ]:
#Finally, move the rest of the data as a training part
data = load_data(path_data)
for i in data:
    shutil.move(i, dir_train)

## Data Augmentation

**Setting up directories**

In [ ]:
#Set path to your split train directory
train_directory = r'C:\Users\grofc\Desktop\Preprocessed\Train'
os.chdir(train_directory)
dirs = os.listdir(train_directory)

print(f"Number of training images: {len(dirs)}")

for dir in dirs:
    img_dir = os.listdir(os.path.join(train_directory, dir))
    print(img_dir)
    print(dirs[0])
    break

**Augmentation parameters**

In [ ]:
#Select number of images to be augmented
number_to_aug = 1
ratio = number_to_aug / len(dirs)

print(f'Img to augment: {number_to_aug}, ratio: {100 *ratio:.3f} %')
print(f'After augmentation number of train imgs: {5 * number_to_aug + len(dirs)}')

In [ ]:
#If # of images to augment is less than length of your training part 
#then choose random of them to augmentate
indices = set()

while len(indices) < number_to_aug:
    num = random.randint(0, len(dirs)-1)
    indices.add(num)

for i,item in enumerate(indices):
    print(i,item)
    break
    
indices = list(indices)
print('Lengt of list is: ', len(indices))

**Augmentation Techniques**
- Rotation (90,180,270 Degrees) to keep its original size
- Flipping (Left, right) 

**Rotation of the images**

In [ ]:
#Function rotates image
#Rotation is done by 90,180, 270 degrees clocwise

def rotate_img(image, rotations=4):
    
    imgs = []
    
    if image is None:
        return
    if rotations > 4:
        rotations = 4
    
    for rt in range(1,rotations):
        img = tf.image.rot90(image, rt)
        img = np.array(img, dtype=np.uint8)
        imgs.append(img)
        
    return imgs

**Flipping of the image**

**Flipping left**

In [ ]:
#Function flips images horizontally

def flip_left(img):

    if img is None:
        return
    
    img = np.array(tf.image.flip_left_right(img))
    
    return img

**Flipping down**

In [ ]:
#Function flips images vertically

def flip_down(img):

    if img is None:
        return
    
    img = np.array(tf.image.flip_up_down(img))
    
    return img

## Augmenation

In [ ]:
# Function augmentates images and saves them to training directory
#Param: dirs - loaded train images names
#Param: indices - Randomly selected indices of train part to be augmented
#Param: train_dir - Path to train directory
#Param: format - Format for images to be saved

def augmentate(dirs, indices, train_dir, format='.png'):
    
    img_path = train_dir + '/img'
    mask_path =  train_dir + '/mask'

    #Iterate over train directory and augment only images within random indices
    for i,index in enumerate(indices):

        name = dirs[index]
        img_path = train_dir + '/' + dirs[index] + '/img/' + dirs[index] + format
        mask_path =  train_dir + '/' + dirs[index] + '/mask/' + dirs[index] + '_mask' + format

       #Load the image/mask
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

        #List of augmented images/mask
        aug_img = []
        aug_mask = []
        
        #Flipping down
        aug_img.append(flip_down(img))
        aug_mask.append(flip_down(mask.reshape(mask.shape[0], mask.shape[1],1)))
        
        #Flipping left
        aug_img.append(flip_left(img))
        aug_mask.append(flip_left(mask.reshape(mask.shape[0], mask.shape[1],1)))
        
        #Rotating:
        aug_img.extend(rotate_img(img))
        aug_mask.extend(rotate_img(mask.reshape(mask.shape[0], mask.shape[1],1)))
        
        #Iterate over the augmented images
        for i in range(0,len(aug_img)):
            try:
                #Make suffix to '_aug_(0-4) for augmented images'
                sufix = '_aug_' + str(i)
                path = train_dir + '/' + name + sufix
                path_img = path + '/img'
                path_mask = path + '/mask' 
                
                #Make new dirs for augmented images
                os.makedirs(path)
                os.makedirs(path_img)
                os.makedirs(path_mask)
                
                #Save the images
                cv2.imwrite(path_img + '/' + name + sufix  + format, aug_img[i] )
                cv2.imwrite(path_mask + '/' + name + sufix + '_mask' + format, aug_mask[i])
            except:
                continue

In [ ]:
#Run to augmentate training part
#Please be patient again....
augmentate(dirs=dirs, indices=indices, train_dir=train_directory)